In [4]:
# imports
import numpy as np
import matplotlib.pyplot as plt


In [5]:
# constants
R = 8.314 

In [14]:
# define Toth model functions
def get_qs(T):
    return qs0 * np.exp(chi * (1 - T / T0)) # eq 9

def get_b(T):
    exponent = (-dH / (R * T0)) * ((T0 / T) - 1)
    return b0 * np.exp(exponent) # eq 10

def get_t(T):
    return t0 + alpha * (1 - (T0 / T)) # eq 11

def toth_isotherm(pressure, T):
    qs_val = get_qs(T)
    b_val = get_b(T)
    t_val = get_t(T)
    
    numerator = qs_val * b_val * pressure
    denominator = (1 + (b_val * pressure)**t_val)**(1 / t_val)
    
    return numerator / denominator # eq 8


# define GAB isotherm equation for H2O
def gab_isotherm(x_h2o, params_gab): # x_h2o is RH from 0 to 1
    q_m, c, k_G = params_gab
    
    numerator = q_m * c * k_G * x_h2o
    
    term1 = 1 - k_G * x_h2o
    term2 = 1 + (c - 1) * k_G * x_h2o
    denominator = term1 * term2
    
    return numerator / denominator # eq 12


# define WADST model
import numpy as np

def wadst_model(P_co2, q_h2o, params_dry, params_wet, A):
    qs_d, b_d, t_d = params_dry
    qs_w, b_w, t_w = params_wet

    if q_h2o > 1e-9:
        weight = np.exp(-A / q_h2o)
    else:
        weight = 0.0

    # dry Toth term
    dry_term = (qs_d * b_d * P_co2) / ((1 + (b_d * P_co2)**t_d)**(1/t_d))

    # wet Toth term
    wet_term = (qs_w * b_w * P_co2) / ((1 + (b_w * P_co2)**t_w)**(1/t_w))

    q_total = (1 - weight) * dry_term + weight * wet_term # eq 13
    
    return q_total


# define CATSO model
def catso_model(P_co2, x_h2o, params_dry, params_wet, water_params):    
    qs_d, b_d, t_d = params_dry
    qs_w, b_w, t_w = params_wet
    b_h2o, S, K, x_m = water_params

    # competitive term (equation 15)
    denom_comp = (1 + (b_d * P_co2)**t_d + (b_h2o * x_h2o)**t_d)**(1/t_d)
    q_comp = (qs_d * b_d * P_co2) / denom_comp

    # theta function (equations 17 and 18)
    w_x = 1 / (1 + np.exp(-S * (x_h2o - x_m))) # eq 17
    theta_m = (1 - w_x) * K * x_h2o + w_x # eq 18

    # Toth isotherm for the wet sites
    denom_wet = (1 + (b_w * P_co2)**t_w)**(1/t_w)
    base_wet_q = (qs_w * b_w * P_co2) / denom_wet
    q_wet = theta_m * base_wet_q # eq 16

    # Total uptake
    q_total = q_comp + q_wet # eq 14
    
    return q_total